In [ ]:
import pandas, os, sys, re, time, collections, Bio, pprint, random, glob

from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio import SeqIO
import numpy as np

In [ ]:
id_mappings = pandas.read_csv('./old_pma_analysis_inputs/id_mappings.txt', sep='\t')
print(id_mappings.head(1))
domains = pandas.read_csv('./old_pma_analysis_inputs/ensg_to_protein_domain.txt', sep='\t')
print(domains.head(2))
ensg_to_domains = collections.defaultdict(set)
for ensg, desc in zip(domains['Ensembl Gene ID'].tolist(), domains['Interpro Short Description'].tolist()):
    ensg_to_domains[ensg].add(desc)

symbol_to_domains = collections.defaultdict(set)
for symbol, ensg in zip(id_mappings['Gene Symbol'], id_mappings['Ensembl Gene ID']):
    symbol_to_domains[symbol] |= ensg_to_domains.get(ensg, set())
print(symbol_to_domains['LRRIQ2'])
dead = {}
for symbol, domains in symbol_to_domains.items():
    if type(symbol) != type(''):
        continue
    #if 'DDX' in symbol:
    #    continue
    #if 'DEAD/DEAH_box_helicase_dom' in domains:
        #dead[symbol] = domains
    #    continue
    for x in [_ for _ in domains if type(_) == type('')]:
        if 'DEAD' in x or 'box_helicase' in x or ('DEXD' in x) or ('RNA_helicase' in x):
            dead[symbol] = domains
print(len(dead))
print(dead)

In [ ]:
amigo = pandas.read_csv('listsOfRbps/rna_binding_genes_from_amigo.txt', sep='\t', comment='#')['Gene name'].tolist()


In [ ]:
df = pandas.read_csv('listsOfRbps/DDX_genes.txt', sep='\t', header=None)
print(len(df.index))
print('\n'.join(df[1]))

ddx = list(df[1]) + list(dead.keys())
print('----')
print('\n'.join([str(x) for x in ddx]))

In [ ]:
omim = pandas.read_csv('old_pma_analysis_inputs/omim/genemap2.txt', sep='\t', skiprows=3, dtype=str)
print(omim.head(2))
omim.dropna(inplace=True, subset=['Phenotypes'])
#y= omim.loc[0, 'Phenotypes']
#print(type(y))
omim = omim.loc[[any([bool(x in amigo) for x in str(cs).split(', ')]) for cs in omim['Gene Symbols']]]
#omim = omim.loc[[bool() for x in omim['Phenotypes']], :]
omim.to_excel('outputs/omim_genemap2_rbps.xlsx')
print(omim.head(3))

In [ ]:
df = pandas.read_csv('cancerLists/humanProteinAtlasPathology.tsv', sep='\t')
df = df.loc[[(x in amigo) for x in df['Gene name']], :]
_df = df.copy()

In [ ]:
df = _df.copy()
print(df['Cancer'].value_counts())
#df = df.loc[[bool(re.search('skin', x, re.IGNORECASE) is not None) for x in df['Cancer']], :]
df = df.loc[df['Cancer']=='head and neck cancer', :]
print(df)
favorable = df.loc[df['prognostic - favourable']<0.01, :]
unfavorable = df.loc[df['prognostic - unfavourable']<0.01, :]
unfavorable.sort_values(by='prognostic - unfavourable', inplace=True)
favorable.sort_values(by='prognostic - favourable', inplace=True)
print('\n'.join(unfavorable['Gene name'].head(10)))

print(unfavorable)
print('\n favor:')
print('\n'.join(favorable['Gene name'].head(10)))

print(favorable)
